# [gpt4o] Plain Text Generation with gpt-4o
This sample demonstrates how to generate a plain text file to train custom speech model. 

> ✨ ***Note*** <br>
> Please check the custom speech support for each language before you get started - https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=stt#:~:text=Custom%20speech%20support 


## Prerequisites
Git clone the repository to your local machine. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

* A subscription key for the Speech service. See [Try the speech service for free](https://docs.microsoft.com/azure/cognitive-services/speech-service/get-started).
* Python 3.5 or later needs to be installed. Downloads are available [here](https://www.python.org/downloads/).
* The Python Speech SDK package is available for Windows (x64 or x86) and Linux (x64; Ubuntu 16.04 or Ubuntu 18.04).
* On Ubuntu 16.04 or 18.04, run the following commands for the installation of required packages:
  ```sh
  sudo apt-get update
  sudo apt-get install libssl1.0.0 libasound2
  ```
* On Debian 9, run the following commands for the installation of required packages:
  ```sh
  sudo apt-get update
  sudo apt-get install libssl1.0.2 libasound2
  ```
* On Windows you need the [Microsoft Visual C++ Redistributable for Visual Studio 2017](https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads) for your platform.

Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

```bash
pip install -r requirements.txt
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.

## Generate synthetic dataset with gpt-4o

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

speech_key = os.getenv("AZURE_AI_SPEECH_API_KEY")
speech_region = os.getenv("AZURE_AI_SPEECH_REGION")

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
    
try:
    client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version
    )
except (ValueError, TypeError) as e:
    print(e)

aoai_deployment_name

In [ ]:
system_message = """
Generate plain text sentences of #topic# related text to improve the recognition of domain-specific words and phrases.
Domain-specific words can be uncommon or made-up words, but their pronunciation must be straightforward to be recognized. 
Use text data that's close to the expected spoken utterances. The nummber of utterances per line should be 1. 
"""

topic = """
LG electronics call center QnA related expected spoken utterances for Vietnamese and English languages.
"""
question = """
create 20 lines of jsonl of the topic in Vietnamese and english. jsonl format is required. use no as number and vi-VN, en-US keys for the languages.
only include the lines as the result. Do not include ```jsonl, ``` and blank line in the result. 
"""

user_message = f"""
#topic#: {topic}
Question: {question}
"""

# Simple API Call
response = client.chat.completions.create(
    model=aoai_deployment_name,
    messages=[
        {"role": "user", "content": user_message},
    ],
  
)
content = response.choices[0].message.content
print(content)
print("Usage Information:")
#print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}") #only o1 models support this
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

In [ ]:
with open('lg_support_expressions.jsonl', 'w', encoding='utf-8') as f:
    f.write(content)

## Create plain text file for training custom speech model

In [ ]:
import datetime

languages = ['vi-VN'] # List of languages to generate audio files
output_dir = "plain_text"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open('cc_support_expressions.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        try:
            expression = json.loads(line)
            for lang in languages:
                text = expression[lang]
                timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                file_name = f"{lang}_{timestamp}.txt"
                with open(os.path.join(output_dir,file_name), 'a', encoding='utf-8') as plain_text:
                    plain_text.write(f"{text}\n")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line}")
            print(e)

In [ ]:
output_dir = "plain_text"  # Redefine output_dir if necessary

with open(os.path.join(output_dir, file_name), 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)